In [2]:
# from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature=0.1)
chat.predict("How many planets are there?")


Expected output: "There are eight recognized planets in our solar system, including Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune." 

In [ ]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(content="You are a geography export and you only reply in French."),
    AIMessage(content="Bonjour, je m'appelle Jungmoo"),
    HumanMessage(content="What is the distance between South Korea and France. Also, what is your name?")
]

chat.predict_messages(messages)


Expected output: "Bonjour! Je m'appelle Jungmoo. La distance entre la Corée du Sud et la France est d'environ 8 500 kilomètres en ligne droite."

In [ ]:
from langchain.prompts import PromptTemplate 

template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}.")
prompt = template.format(country_a="South Korea", country_b="France")
chat.predict(prompt)


Expected output: "The distance between South Korea and France is approximately 8,500 kilometers (5,280 miles)."

In [ ]:
from langchain.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography export and you only reply in {language}."),
    ("ai", "Bonjour, je m'appelle {name}"),
    ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?")
])

prompt = template.format_messages(
    language="French",
    name="Cheon",
    country_a="South Korea",
    country_b="France"
)

chat.predict_messages(prompt)


Expected output: "Bonjour! Je m'appelle Cheon. La distance entre la Corée du Sud et la France est d'environ 8 500 kilomètres en ligne droite."

In [ ]:
#OutputParser
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do not reply anything else."),
    ("human", "{question}")
])

prompt = template.format_messages(max_items=10, question="what are the colors?")
result = chat.predict_messages(prompt)

p = CommaOutputParser()
p.parse(result.content)


Expected output: ["red", "green", "indigo", "blue", "pink", "black", "white", "gray", "violet", "yellow"]

In [ ]:
# Langchain expression language
# chain : Prompt Template | Chat Model | Output Parser
chain = template | chat | CommaOutputParser()
chain.invoke({
    "max_items":5,
    "queation":"What are pokemons?"
})

Expected output: ["pikachu", "charizard", "bulbasaur", "squirtle", "jigglypuff"]

In [ ]:
# Multiple chains
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler])
chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients"),
    ("human", "I want to cook {cuisine} food.")
])

chef_chain = chef_prompt | chat

veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", {recipe})
])

veg_chef_chain = veg_chef_prompt | chat

final_chain = {"recipe":chef_chain} | veg_chef_chain

final_chain.invoke({
    "cuisine":"indian"
})

Expected output:</br>
      That's great! Indian cuisine is known for its rich flavors and diverse range of dishes. Here's a simple recipe for you to try:</br>
      Butter Chicken (Murgh Makhani):
      Ingredients:</br>
      - 500g boneless chicken cut into pieces</br>
      - 2 tablespoons butter</br>
      - 1 onion finely chopped</br>
      - 2 teaspoons ginger-garlic paste</br>
      - 2 teaspoons red chili powder</br>
      - 1 teaspoon turmeric powder</br>
      - 1 teaspoon garam masala</br>
      - 1 cup tomato puree</br>
      - 1/2 cup heavy cream</br>
      - Salt to taste</br>
      - Fresh coriander leaves for garnish</br>
      </br>
      Instructions:</br>
      1. Heat butter in a pan and add the chopped onions. Sauté until they turn golden brown.</br>
      2. Add ginger-garlic paste and cook for a minute until the raw smell disappears.</br>
      3. Add red chili powder, turmeric powder, and garam masala. Mix well.</br>
      4. Add tomato puree and cook for 5-7 minutes until the oil separates from the masala.</br>
      5. Add the chicken pieces and cook until they are no longer pink.</br>
      6. Pour in the heavy cream and mix well. Simmer for 10-15 minutes until the chicken is cooked through and the sauce thickens.</br>
      7. Adjust the salt according to your taste.</br>
      8. Garnish with fresh coriander leaves and serve hot with naan or rice.</br>
      </br>
      Remember, Indian cuisine offers a wide variety of dishes, so feel free to explore and experiment with different recipes and flavors. Enjoy your cooking!Butter Chicken (Murgh Makhani) is a popular Indian dish known for its rich and creamy tomato-based sauce. To make this recipe vegetarian, you can replace the chicken with a suitable alternative such as tofu or paneer (Indian cottage cheese). Here's how you can modify the recipe:</br>
      </br>
      Ingredients:</br>
      - 500g tofu or paneer, cut into pieces</br>
      - 2 tablespoons butter (or vegan butter for a vegan version)</br>
      - 1 onion, finely chopped</br>
      - 2 teaspoons ginger-garlic paste</br>
      - 2 teaspoons red chili powder</br>
      - 1 teaspoon turmeric powder</br>
      - 1 teaspoon garam masala</br>
      - 1 cup tomato puree</br>
      - 1/2 cup heavy cream (or coconut cream for a vegan version)</br>
      - Salt to taste</br>
      - Fresh coriander leaves for garnish</br>
      </br>
      Instructions:</br>
      1. Heat butter in a pan and add the chopped onions. Sauté until they turn golden brown.</br>
      2. Add ginger-garlic paste and cook for a minute until the raw smell disappears.</br>
      3. Add red chili powder, turmeric powder, and garam masala. Mix well.</br>
      4. Add tomato puree and cook for 5-7 minutes until the oil separates from the masala.</br>
      5. Add the tofu or paneer pieces and cook until they are heated through.</br>
      6. Pour in the heavy cream (or coconut cream) and mix well. Simmer for 10-15 minutes until the sauce thickens.</br>
      7. Adjust the salt according to your taste.</br>
      8. Garnish with fresh coriander leaves and serve hot with naan or rice.</br>
      </br>
      By substituting the chicken with tofu or paneer, you can still enjoy the flavors and textures of this classic Indian dish while keeping it vegetarian.

,[
       "AIMessageChunk(content=\"Butter Chicken (Murgh Makhani) is a popular Indian dish known for its rich and creamy tomato-based sauce. To make this recipe vegetarian, you can replace the chicken with a suitable alternative such as tofu or paneer (Indian cottage cheese). Here's how you can modify the recipe:\\n\\nIngredients:\\n- 500g tofu or paneer, cut into pieces\\n- 2 tablespoons butter (or vegan butter for a vegan version)\\n- 1 onion, finely chopped\\n- 2 teaspoons ginger-garlic paste\\n- 2 teaspoons red chili powder\\n- 1 teaspoon turmeric powder\\n- 1 teaspoon garam masala\\n- 1 cup tomato puree\\n- 1/2 cup heavy cream (or coconut cream for a vegan version)\\n- Salt to taste\\n- Fresh coriander leaves for garnish\\n\\nInstructions:\\n1. Heat butter in a pan and add the chopped onions. Sauté until they turn golden brown.\\n2. Add ginger-garlic paste and cook for a minute until the raw smell disappears.\\n3. Add red chili powder, turmeric powder, and garam masala. Mix well.\\n4. Add tomato puree and cook for 5-7 minutes until the oil separates from the masala.\\n5. Add the tofu or paneer pieces and cook until they are heated through.\\n6. Pour in the heavy cream (or coconut cream) and mix well. Simmer for 10-15 minutes until the sauce thickens.\\n7. Adjust the salt according to your taste.\\n8. Garnish with fresh coriander leaves and serve hot with naan or rice.\\n\\nBy substituting the chicken with tofu or paneer, you can still enjoy the flavors and textures of this classic Indian dish while keeping it vegetarian.\")"
      ]